In [28]:
import pandas as pd
import Levenshtein
import math

In [2]:
pd.options.display.max_columns = 22
pd.options.display.max_rows = 400

### Read in all taxes data + prelim cleaning

In [3]:
df_taxes_list = []
path = "~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/"
for i in range(2009, 2020):
    df_taxes_list.append(pd.read_csv(path+"AY"+str(i)+ " Real Property_Personal Property.csv"))
df_full_taxes = pd.concat(df_taxes_list, ignore_index=True)
df_full_taxes.head();

/anaconda3/envs/cds/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/envs/cds/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (6,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_full_taxes.shape

(421915, 21)

In [5]:
df_home = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/fixed_home_v02.csv')
df_cdbg = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/fixed_cdbg.csv')

In [6]:
def genStrip():
    return lambda addr: addr.strip()

In [37]:
def genSpacing():
    def test(word):
        if word == "  ":
            return ""
        elif type(word) == float:
            if math.isnan(word):
                return ""
        else:
            return word
    return test

In [38]:
df_full_taxes['FULL_ADDRESS'] = df_full_taxes['HOUSE_NO'].astype(str).apply(genStrip()) + \
' ' + df_full_taxes['STDIRECT'].apply(genSpacing()) + df_full_taxes["STREET_NAM"].apply(genStrip()) + ' ' + df_full_taxes["STREET_TYPE"].astype(str).apply(genStrip())

In [40]:
df_full_taxes.head(1000)

,OWNER_NAME,HOUSE_NO,STDIRECT,STREET_NAM,STREET_TYPE,UNIT,PROP_ZIP,ZONE_CODE,SUBDIVISION,PARCEL_NO,PREV_VAL,CURR_VAL,VALCHGDATE,TAXDISTRIC,HOMEEXEMPT,TOTALACRES,PROPERTY_CLASS,DIG_STRAT,BUSI_ID,NAICS,PERSKEY,FULL_ADDRESS
0,NICHOLS DORRIS WILDER,0,NaN,A C L RAILROAD,NaN,NaN,NaN,C3,COUNTY LINE - ACREE AREA,00104/00001/014,13100,13100,11/12/2008,2,S0,0.00,RESIDENTIAL,3,NaN,NaN,NaN,0 A C L RAILROAD nan
1,WESTON MONTY,0,,A C L RAILROAD,NaN,NaN,NaN,C3,COUNTY LINE - ACREE AREA,00104/00001/015,26300,26300,11/12/2008,2,S0,0.39,COMMERCIAL,3,NaN,NaN,NaN,0 A C L RAILROAD nan
2,NICHOLS JOE,0,,A C L RAILROAD,NaN,NaN,NaN,C3,COUNTY LINE - ACREE AREA,00104/00001/016,990,990,11/12/2008,2,S0,0.25,COMMERCIAL,3,NaN,NaN,NaN,0 A C L RAILROAD nan
3,ISRAEL CAROLYN R ETAL,0,NaN,A C L RAILROAD,NaN,NaN,NaN,AG,MASSEY,00155/00004/013,21400,21400,4/16/2007,1,S0,17.19,RESIDENTIAL,4,NaN,NaN,NaN,0 A C L RAILROAD nan
4,GEER RET,0,NaN,A C L RAILROAD,NaN,NaN,NaN,AG,LIBERTY EXPRESSWAY INDUSTRIAL AREA,00186/00001/006,28600,28600,6/20/2007,1,S0,14.30,RESIDENTIAL,4,NaN,NaN,NaN,0 A C L RAILROAD nan
5,WILDER SARAH P & DORRIS,0,NaN,A C L RAILROAD,NaN,NaN,NaN,R4,M. L. WILDER,00104/00002/031,3200,3200,NaN,2,S0,0.21,RESIDENTIAL,3,NaN,NaN,NaN,0 A C L RAILROAD nan
6,WILDER SARAH P & DORRIS,0,NaN,A C L RAILROAD,NaN,NaN,NaN,R4,M. L. WILDER,00104/00002/030,4300,4300,NaN,2,S0,0.21,RESIDENTIAL,3,NaN,NaN,NaN,0 A C L RAILROAD nan
7,HENDERSON PEARL WILDER,0,NaN,A C L RAILROAD,NaN,NaN,NaN,R4,M. L. WILDER,00104/00002/004,3000,3000,3/6/2009,2,S0,0.17,RESIDENTIAL,3,NaN,NaN,NaN,0 A C L RAILROAD nan
8,WILDER SARAH P & DORRIS,0,NaN,A C L RAILROAD,NaN,NaN,NaN,R6,COUNTY LINE - ACREE AREA,00104/00001/013,4000,4000,11/12/2008,2,S0,1.05,RESIDENTIAL,3,NaN,NaN,NaN,0 A C L RAILROAD nan
9,HENDERSON PEARL WILDER,0,NaN,A C L RAILROAD,NaN,NaN,NaN,R4,M. L. WILDER,00104/00002/003,9400,9400,10/31/2008,2,S0,0.22,RESIDENTIAL,3,NaN,NaN,NaN,0 A C L RAILROAD nan


In [8]:
df_house = pd.concat([df_home, df_cdbg], ignore_index = True)
df_house.drop('Unnamed: 0', axis = 1, inplace = True)
s_house = pd.Series(df_house['Address'].unique()) # get unique values for housing proj addresses

In [9]:
s_house.shape

(670,)

### Read in addr junction table list and compare

In [52]:
df_addr = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/ToDatabase/addr_junct_table.csv')

In [53]:
f_addresses = df_addr['Address']
t_addresses = pd.Series(df_full_taxes['FULL_ADDRESS'].unique()) # to compare

In [54]:
t_addresses[~t_addresses.isin(f_addresses)].shape

(16224,)

In [55]:
no_tax = f_addresses[~f_addresses.isin(t_addresses)] # addresses that we "don't" have tax data for

In [66]:
no_tax.head(15)

4           509 W GORDON AVE
5           610 W GORDON AVE
6           700 W GORDON AVE
7           708 W GORDON AVE
23             2632 ERICA CT
30        235 BONNY VIEW AVE
33              1200 AUGUSTA
37          205 S COLLINS ST
46          1705 E BROAD AVE
50    1001 RADIUM SPRINGS RD
56           109 W BROAD AVE
57             110 SHELBY LN
58             112 SHELBY LN
65        252 BONNY VIEW AVE
66        243 BONNY VIEW AVE
Name: Address, dtype: object

In [57]:
def lev_list(address_list, address, i): # will help us find near matches
    temp_list = []
    temp_list_u = []
    for u_address in address_list:
        if Levenshtein.distance(address, u_address.upper()) <= i:
            temp_list.append(address)
            temp_list_u.append(u_address.upper())
    temp_list_series = pd.Series(temp_list)
    temp_list_u_series = pd.Series(temp_list_u)
    return temp_list_u_series

In [72]:
lev_list(t_addresses, "205 COLLINS AVE", 4) # some exploratory checking

0      805 COLLEY AVE
1     106 COLLINS AVE
2     108 COLLINS AVE
3     109 COLLINS AVE
4     110 COLLINS AVE
5     111 COLLINS AVE
6     112 COLLINS AVE
7     113 COLLINS AVE
8     200 COLLINS AVE
9     201 COLLINS AVE
10    202 COLLINS AVE
11    203 COLLINS AVE
12    204 COLLINS AVE
13    205 COLLINS AVE
14    206 COLLINS AVE
15    207 COLLINS AVE
16    208 COLLINS AVE
17    209 COLLINS AVE
18    210 COLLINS AVE
19    211 COLLINS AVE
20    300 COLLINS AVE
21    301 COLLINS AVE
22    302 COLLINS AVE
23    304 COLLINS AVE
24    306 COLLINS AVE
25    308 COLLINS AVE
26      205 FLINT AVE
dtype: object

In [33]:
df_full_taxes['STDIRECT'].value_counts().index

Index(['  ', 'S ', 'N ', 'E ', 'W ', 'SE', 'NE', 'R '], dtype='object')

In [36]:
df_full_taxes['STDIRECT'][0]

float

In [51]:
df_full_taxes[df_full_taxes['STREET_NAM'].str.contains("MADISON")]

,OWNER_NAME,HOUSE_NO,STDIRECT,STREET_NAM,STREET_TYPE,UNIT,PROP_ZIP,ZONE_CODE,SUBDIVISION,PARCEL_NO,PREV_VAL,CURR_VAL,VALCHGDATE,TAXDISTRIC,HOMEEXEMPT,TOTALACRES,PROPERTY_CLASS,DIG_STRAT,BUSI_ID,NAICS,PERSKEY,FULL_ADDRESS
19617,MADISON PARK DEVELOPMENT INC,0,S,MADISON,ST,NaN,NaN,R1,MADISON PARK,00012/00003/21E,3500,3500,10/16/2007,1,S0,0.07,RESIDENTIAL,3,NaN,NaN,NaN,0 S MADISON ST
19618,MADISON PARK DEVELOPMENT INC,0,S,MADISON,ST,NaN,NaN,R1,MADISON PARK,00012/00003/21F,3500,3500,10/16/2007,1,S0,0.07,RESIDENTIAL,3,NaN,NaN,NaN,0 S MADISON ST
19619,MADISON PARK DEVELOPMENT INC,0,S,MADISON,ST,NaN,NaN,R1,MADISON PARK,00012/00003/21A,3500,3500,10/16/2007,1,S0,0.07,RESIDENTIAL,3,NaN,NaN,NaN,0 S MADISON ST
19620,MADISON PARK DEVELOPMENT INC,0,S,MADISON,ST,NaN,NaN,R1,MADISON PARK,00012/00003/21D,7500,7500,10/1/2008,1,S0,0.16,RESIDENTIAL,3,NaN,NaN,NaN,0 S MADISON ST
19621,ALBANY GA CITY OF DCED,0,S,MADISON,ST,NaN,NaN,R3,RIVER ROAD,00012/00003/20B,730,730,10/16/2007,1,S0,0.23,NaN,1,NaN,NaN,NaN,0 S MADISON ST
19622,THOMAS ROBERT D,0,S,MADISON,ST,NaN,NaN,R3,RIVER ROAD,00012/00002/52C,1700,1700,3/14/2007,1,S0,0.42,RESIDENTIAL,3,NaN,NaN,NaN,0 S MADISON ST
19623,ALBANY GA CITY OF DCED,0,S,MADISON,ST,NaN,NaN,R3,RIVER ROAD,00012/00003/20E,730,730,10/16/2007,1,S0,0.23,NaN,1,NaN,NaN,NaN,0 S MADISON ST
19624,ALBANY GA CITY OF DCED,0,S,MADISON,ST,NaN,NaN,R3,RIVER ROAD,00012/00003/20C,730,730,10/16/2007,1,S0,0.23,NaN,1,NaN,NaN,NaN,0 S MADISON ST
19625,ALBANY GA CITY OF DCED,0,S,MADISON,ST,NaN,NaN,R3,RIVER ROAD,00012/00003/20D,830,830,10/16/2007,1,S0,0.32,NaN,1,NaN,NaN,NaN,0 S MADISON ST
19626,THOMAS JOSEPHINE,0,S,MADISON,ST,NaN,NaN,R3,RIVER ROAD,00012/00003/15C,1000,1000,10/16/2007,1,S0,0.13,RESIDENTIAL,3,NaN,NaN,NaN,0 S MADISON ST
